In [1]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests
import tqdm
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import datetime

# Requests
import requests

from time import sleep
import random

# Para trabajar con archivos dotenv y los tokens
# -----------------------------------------------------------------------
import os
import dotenv
dotenv.load_dotenv(dotenv_path="../src/.env")
# Para Importar desde rutas recursivas
# -----------------------------------------------------------------------
import sys
sys.path.append("../") #Para poder importar
from src.SupportVuelos import get_airport

# Aeropuertos
- MAD: 95565077
- OSL:
- NCE:

In [5]:
aeropuertos = ["madrid","niza","oslo"]
resultados = [get_airport(i) for i in aeropuertos]

In [12]:
resultados
df = pd.DataFrame()
for i in range(0,len(resultados)):
    df_temp = pd.DataFrame({
        "skyId": [resultados[i][0]],
        "entityId": [resultados[i][1]],
        "flightPlaceType" : [resultados[i][2]],
        "localizedName" : [resultados[i][3]]})
    df = pd.concat([df,df_temp],ignore_index=True)

df

,skyId,entityId,flightPlaceType,localizedName
0,MAD,95565077,AIRPORT,Madrid-Barajas
1,NCE,95673860,AIRPORT,Niza
2,OSL,128667763,AIRPORT,Oslo-Gardermoen


In [13]:
df.to_csv("../datos/aeropuertos.csv")

In [2]:
df = pd.read_csv("../datos/aeropuertos.csv",index_col="Unnamed: 0")

In [3]:
df

,skyId,entityId,flightPlaceType,localizedName
0,MAD,95565077,AIRPORT,Madrid-Barajas
1,NCE,95673860,AIRPORT,Niza
2,OSL,128667763,AIRPORT,Oslo-Gardermoen


In [63]:
skyId = df["skyId"].tolist()
entityId = df["entityId"].tolist()
flightPlaceType = df["flightPlaceType"].tolist()
localizedName = df["localizedName"].tolist()
cabinclass = ["economy","premium_economy","business","first"]

In [90]:
querystrings = []

for i in range(1,len(entityId)):
    for date in dates:
        for clase in cabinclass:
            querystring = {
                "originSkyId":skyId[0],
                "destinationSkyId":skyId[i],
                "originEntityId": entityId[0],
                "destinationEntityId": entityId[i],
                "date":date[0],
                "returnDate":date[1],
                "cabinClass":clase,
                "adults":"4",
                "sortBy":"best",
                "limit" : "10",
                "currency":"EUR",
                "market":"es-ES",
                "countryCode":"ES"
                }
                
            querystrings.append(querystring)

querystrings[0]["cabinClass"]

'economy'

In [91]:
len(querystrings)//2

20

In [83]:
querystrings[20]

{'originSkyId': 'MAD',
 'destinationSkyId': 'OSL',
 'originEntityId': 95565077,
 'destinationEntityId': 128667763,
 'date': '2024-10-24',
 'returnDate': '2024-10-27',
 'cabinClass': 'economy',
 'adults': '4',
 'sortBy': 'best',
 'limit': '10',
 'currency': 'EUR',
 'market': 'es-ES',
 'countryCode': 'ES'}

In [9]:
results = {
    "status": True,
    "message": "success",
    "data": {
        "context": {
            "status": "incomplete",
            "sessionId": "ClQIARJQCk4KJGY4NGFlOTA2LTZhNjgtNDM2MC05YTZhLTg0NGViYjA3NzUwOBACGiRhODQwMjc5OC04Nzg5LTRmYWMtYWYxMC1iMDA5MmI2MmQzZGISKHVzc19jYjI3YjQyMy05NGYyLTQ2NWQtOGI0NC05NDFkOWFkNmVhYmY=",
            "totalResults": 10
        },
        "itineraries": [
            {
                "id": "13870-2410241015--32222-0-14581-2410241210|14581-2410271900--32222-0-13870-2410272100",
                "price": {
                    "raw": 1969.04,
                    "formatted": "1,970 €",
                    "pricingOptionId": "KDI3UJAlq78x"
                },
                "legs": [
                    {
                        "id": "13870-2410241015--32222-0-14581-2410241210",
                        "origin": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 115,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-24T10:15:00",
                        "arrival": "2024-10-24T12:10:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32222,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/IB.png",
                                    "name": "Iberia"
                                }
                            ],
                            "operating": [
                                {
                                    "id": -32639,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/YW.png",
                                    "name": "Air Nostrum"
                                }
                            ],
                            "operationType": "not_operated"
                        },
                        "segments": [
                            {
                                "id": "13870-14581-2410241015-2410241210--32222",
                                "origin": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "destination": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "departure": "2024-10-24T10:15:00",
                                "arrival": "2024-10-24T12:10:00",
                                "durationInMinutes": 115,
                                "flightNumber": "8726",
                                "marketingCarrier": {
                                    "id": -32222,
                                    "name": "Iberia",
                                    "alternateId": "IB",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32639,
                                    "name": "Air Nostrum",
                                    "alternateId": "YW",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    },
                    {
                        "id": "14581-2410271900--32222-0-13870-2410272100",
                        "origin": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 120,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-27T19:00:00",
                        "arrival": "2024-10-27T21:00:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32222,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/IB.png",
                                    "name": "Iberia"
                                }
                            ],
                            "operating": [
                                {
                                    "id": -32639,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/YW.png",
                                    "name": "Air Nostrum"
                                }
                            ],
                            "operationType": "not_operated"
                        },
                        "segments": [
                            {
                                "id": "14581-13870-2410271900-2410272100--32222",
                                "origin": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "destination": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "departure": "2024-10-27T19:00:00",
                                "arrival": "2024-10-27T21:00:00",
                                "durationInMinutes": 120,
                                "flightNumber": "1214",
                                "marketingCarrier": {
                                    "id": -32222,
                                    "name": "Iberia",
                                    "alternateId": "IB",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32639,
                                    "name": "Air Nostrum",
                                    "alternateId": "YW",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    }
                ],
                "isSelfTransfer": False,
                "isProtectedSelfTransfer": False,
                "farePolicy": {
                    "isChangeAllowed": False,
                    "isPartiallyChangeable": False,
                    "isCancellationAllowed": False,
                    "isPartiallyRefundable": False
                },
                "eco": {
                    "ecoContenderDelta": 27.515114
                },
                "fareAttributes": {},
                "tags": [
                    "third_shortest"
                ],
                "isMashUp": False,
                "hasFlexibleOptions": False,
                "score": 0.999
            },
            {
                "id": "13870-2410241720--32222-0-14581-2410241915|14581-2410271900--32222-0-13870-2410272100",
                "price": {
                    "raw": 2035.29,
                    "formatted": "2,036 €",
                    "pricingOptionId": "9rzFknTMAxKb"
                },
                "legs": [
                    {
                        "id": "13870-2410241720--32222-0-14581-2410241915",
                        "origin": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 115,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-24T17:20:00",
                        "arrival": "2024-10-24T19:15:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32222,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/IB.png",
                                    "name": "Iberia"
                                }
                            ],
                            "operating": [
                                {
                                    "id": -32639,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/YW.png",
                                    "name": "Air Nostrum"
                                }
                            ],
                            "operationType": "not_operated"
                        },
                        "segments": [
                            {
                                "id": "13870-14581-2410241720-2410241915--32222",
                                "origin": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "destination": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "departure": "2024-10-24T17:20:00",
                                "arrival": "2024-10-24T19:15:00",
                                "durationInMinutes": 115,
                                "flightNumber": "8730",
                                "marketingCarrier": {
                                    "id": -32222,
                                    "name": "Iberia",
                                    "alternateId": "IB",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32639,
                                    "name": "Air Nostrum",
                                    "alternateId": "YW",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    },
                    {
                        "id": "14581-2410271900--32222-0-13870-2410272100",
                        "origin": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 120,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-27T19:00:00",
                        "arrival": "2024-10-27T21:00:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32222,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/IB.png",
                                    "name": "Iberia"
                                }
                            ],
                            "operating": [
                                {
                                    "id": -32639,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/YW.png",
                                    "name": "Air Nostrum"
                                }
                            ],
                            "operationType": "not_operated"
                        },
                        "segments": [
                            {
                                "id": "14581-13870-2410271900-2410272100--32222",
                                "origin": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "destination": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "departure": "2024-10-27T19:00:00",
                                "arrival": "2024-10-27T21:00:00",
                                "durationInMinutes": 120,
                                "flightNumber": "1214",
                                "marketingCarrier": {
                                    "id": -32222,
                                    "name": "Iberia",
                                    "alternateId": "IB",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32639,
                                    "name": "Air Nostrum",
                                    "alternateId": "YW",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    }
                ],
                "isSelfTransfer": False,
                "isProtectedSelfTransfer": False,
                "farePolicy": {
                    "isChangeAllowed": False,
                    "isPartiallyChangeable": False,
                    "isCancellationAllowed": False,
                    "isPartiallyRefundable": False
                },
                "eco": {
                    "ecoContenderDelta": 27.515114
                },
                "fareAttributes": {},
                "isMashUp": False,
                "hasFlexibleOptions": False,
                "score": 0.875323
            },
            {
                "id": "13870-2410240925--31685-1-14581-2410241545|14581-2410271430--32356-0-13870-2410271630",
                "price": {
                    "raw": 940.6,
                    "formatted": "941 €",
                    "pricingOptionId": "_ocOcwrHHbQH"
                },
                "legs": [
                    {
                        "id": "13870-2410240925--31685-1-14581-2410241545",
                        "origin": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 380,
                        "stopCount": 1,
                        "isSmallestStops": False,
                        "departure": "2024-10-24T09:25:00",
                        "arrival": "2024-10-24T15:45:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -31685,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/07.png",
                                    "name": "Vueling Airlines"
                                }
                            ],
                            "operationType": "fully_operated"
                        },
                        "segments": [
                            {
                                "id": "13870-9772-2410240925-2410241045--31685",
                                "origin": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "destination": {
                                    "flightPlaceId": "BCN",
                                    "displayCode": "BCN",
                                    "parent": {
                                        "flightPlaceId": "BARC",
                                        "displayCode": "BCN",
                                        "name": "Barcelona",
                                        "type": "City"
                                    },
                                    "name": "Barcelona",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "departure": "2024-10-24T09:25:00",
                                "arrival": "2024-10-24T10:45:00",
                                "durationInMinutes": 80,
                                "flightNumber": "1003",
                                "marketingCarrier": {
                                    "id": -31685,
                                    "name": "Vueling Airlines",
                                    "alternateId": "07",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -31685,
                                    "name": "Vueling Airlines",
                                    "alternateId": "07",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            },
                            {
                                "id": "9772-14581-2410241420-2410241545--31685",
                                "origin": {
                                    "flightPlaceId": "BCN",
                                    "displayCode": "BCN",
                                    "parent": {
                                        "flightPlaceId": "BARC",
                                        "displayCode": "BCN",
                                        "name": "Barcelona",
                                        "type": "City"
                                    },
                                    "name": "Barcelona",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "destination": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "departure": "2024-10-24T14:20:00",
                                "arrival": "2024-10-24T15:45:00",
                                "durationInMinutes": 85,
                                "flightNumber": "1521",
                                "marketingCarrier": {
                                    "id": -31685,
                                    "name": "Vueling Airlines",
                                    "alternateId": "07",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -31685,
                                    "name": "Vueling Airlines",
                                    "alternateId": "07",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    },
                    {
                        "id": "14581-2410271430--32356-0-13870-2410271630",
                        "origin": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 120,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-27T14:30:00",
                        "arrival": "2024-10-27T16:30:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32356,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/EZ.png",
                                    "name": "easyJet"
                                }
                            ],
                            "operationType": "fully_operated"
                        },
                        "segments": [
                            {
                                "id": "14581-13870-2410271430-2410271630--32356",
                                "origin": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "destination": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "departure": "2024-10-27T14:30:00",
                                "arrival": "2024-10-27T16:30:00",
                                "durationInMinutes": 120,
                                "flightNumber": "1703",
                                "marketingCarrier": {
                                    "id": -32356,
                                    "name": "easyJet",
                                    "alternateId": "EZ",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32356,
                                    "name": "easyJet",
                                    "alternateId": "EZ",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    }
                ],
                "isSelfTransfer": False,
                "isProtectedSelfTransfer": False,
                "farePolicy": {
                    "isChangeAllowed": False,
                    "isPartiallyChangeable": False,
                    "isCancellationAllowed": False,
                    "isPartiallyRefundable": False
                },
                "eco": {
                    "ecoContenderDelta": 30.157948
                },
                "fareAttributes": {},
                "tags": [
                    "cheapest"
                ],
                "isMashUp": True,
                "hasFlexibleOptions": False,
                "score": 0.830746
            },
            {
                "id": "13870-2410242210--32222-0-14581-2410250005|14581-2410271430--32356-0-13870-2410271630",
                "price": {
                    "raw": 1553.13,
                    "formatted": "1,554 €",
                    "pricingOptionId": "fsMxdLq1-rdv"
                },
                "legs": [
                    {
                        "id": "13870-2410242210--32222-0-14581-2410250005",
                        "origin": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 115,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-24T22:10:00",
                        "arrival": "2024-10-25T00:05:00",
                        "timeDeltaInDays": 1,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32222,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/IB.png",
                                    "name": "Iberia"
                                }
                            ],
                            "operating": [
                                {
                                    "id": -32639,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/YW.png",
                                    "name": "Air Nostrum"
                                }
                            ],
                            "operationType": "not_operated"
                        },
                        "segments": [
                            {
                                "id": "13870-14581-2410242210-2410250005--32222",
                                "origin": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "destination": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "departure": "2024-10-24T22:10:00",
                                "arrival": "2024-10-25T00:05:00",
                                "durationInMinutes": 115,
                                "flightNumber": "8732",
                                "marketingCarrier": {
                                    "id": -32222,
                                    "name": "Iberia",
                                    "alternateId": "IB",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32639,
                                    "name": "Air Nostrum",
                                    "alternateId": "YW",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    },
                    {
                        "id": "14581-2410271430--32356-0-13870-2410271630",
                        "origin": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 120,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-27T14:30:00",
                        "arrival": "2024-10-27T16:30:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32356,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/EZ.png",
                                    "name": "easyJet"
                                }
                            ],
                            "operationType": "fully_operated"
                        },
                        "segments": [
                            {
                                "id": "14581-13870-2410271430-2410271630--32356",
                                "origin": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "destination": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "departure": "2024-10-27T14:30:00",
                                "arrival": "2024-10-27T16:30:00",
                                "durationInMinutes": 120,
                                "flightNumber": "1703",
                                "marketingCarrier": {
                                    "id": -32356,
                                    "name": "easyJet",
                                    "alternateId": "EZ",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32356,
                                    "name": "easyJet",
                                    "alternateId": "EZ",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    }
                ],
                "isSelfTransfer": False,
                "isProtectedSelfTransfer": False,
                "farePolicy": {
                    "isChangeAllowed": False,
                    "isPartiallyChangeable": False,
                    "isCancellationAllowed": False,
                    "isPartiallyRefundable": False
                },
                "eco": {
                    "ecoContenderDelta": 32.86617
                },
                "fareAttributes": {},
                "tags": [
                    "shortest"
                ],
                "isMashUp": False,
                "hasFlexibleOptions": False,
                "score": 0.820266
            },
            {
                "id": "13870-2410242210--32222-0-14581-2410250005|14581-2410271900--32222-0-13870-2410272100",
                "price": {
                    "raw": 1792.37,
                    "formatted": "1,793 €",
                    "pricingOptionId": "zeo1upmhne11"
                },
                "legs": [
                    {
                        "id": "13870-2410242210--32222-0-14581-2410250005",
                        "origin": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 115,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-24T22:10:00",
                        "arrival": "2024-10-25T00:05:00",
                        "timeDeltaInDays": 1,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32222,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/IB.png",
                                    "name": "Iberia"
                                }
                            ],
                            "operating": [
                                {
                                    "id": -32639,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/YW.png",
                                    "name": "Air Nostrum"
                                }
                            ],
                            "operationType": "not_operated"
                        },
                        "segments": [
                            {
                                "id": "13870-14581-2410242210-2410250005--32222",
                                "origin": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "destination": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "departure": "2024-10-24T22:10:00",
                                "arrival": "2024-10-25T00:05:00",
                                "durationInMinutes": 115,
                                "flightNumber": "8732",
                                "marketingCarrier": {
                                    "id": -32222,
                                    "name": "Iberia",
                                    "alternateId": "IB",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32639,
                                    "name": "Air Nostrum",
                                    "alternateId": "YW",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    },
                    {
                        "id": "14581-2410271900--32222-0-13870-2410272100",
                        "origin": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 120,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-27T19:00:00",
                        "arrival": "2024-10-27T21:00:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32222,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/IB.png",
                                    "name": "Iberia"
                                }
                            ],
                            "operating": [
                                {
                                    "id": -32639,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/YW.png",
                                    "name": "Air Nostrum"
                                }
                            ],
                            "operationType": "not_operated"
                        },
                        "segments": [
                            {
                                "id": "14581-13870-2410271900-2410272100--32222",
                                "origin": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "destination": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "departure": "2024-10-27T19:00:00",
                                "arrival": "2024-10-27T21:00:00",
                                "durationInMinutes": 120,
                                "flightNumber": "1214",
                                "marketingCarrier": {
                                    "id": -32222,
                                    "name": "Iberia",
                                    "alternateId": "IB",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32639,
                                    "name": "Air Nostrum",
                                    "alternateId": "YW",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    }
                ],
                "isSelfTransfer": False,
                "isProtectedSelfTransfer": False,
                "farePolicy": {
                    "isChangeAllowed": False,
                    "isPartiallyChangeable": False,
                    "isCancellationAllowed": False,
                    "isPartiallyRefundable": False
                },
                "eco": {
                    "ecoContenderDelta": 27.515114
                },
                "fareAttributes": {},
                "tags": [
                    "second_shortest"
                ],
                "isMashUp": False,
                "hasFlexibleOptions": False,
                "score": 0.769794
            },
            {
                "id": "13870-2410241015--32222-0-14581-2410241210|14581-2410271235--32222-0-13870-2410271435",
                "price": {
                    "raw": 2171.48,
                    "formatted": "2,172 €",
                    "pricingOptionId": "-16NPwEGYPV7"
                },
                "legs": [
                    {
                        "id": "13870-2410241015--32222-0-14581-2410241210",
                        "origin": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 115,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-24T10:15:00",
                        "arrival": "2024-10-24T12:10:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32222,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/IB.png",
                                    "name": "Iberia"
                                }
                            ],
                            "operating": [
                                {
                                    "id": -32639,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/YW.png",
                                    "name": "Air Nostrum"
                                }
                            ],
                            "operationType": "not_operated"
                        },
                        "segments": [
                            {
                                "id": "13870-14581-2410241015-2410241210--32222",
                                "origin": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "destination": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "departure": "2024-10-24T10:15:00",
                                "arrival": "2024-10-24T12:10:00",
                                "durationInMinutes": 115,
                                "flightNumber": "8726",
                                "marketingCarrier": {
                                    "id": -32222,
                                    "name": "Iberia",
                                    "alternateId": "IB",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32639,
                                    "name": "Air Nostrum",
                                    "alternateId": "YW",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    },
                    {
                        "id": "14581-2410271235--32222-0-13870-2410271435",
                        "origin": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 120,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-27T12:35:00",
                        "arrival": "2024-10-27T14:35:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32222,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/IB.png",
                                    "name": "Iberia"
                                }
                            ],
                            "operating": [
                                {
                                    "id": -32639,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/YW.png",
                                    "name": "Air Nostrum"
                                }
                            ],
                            "operationType": "not_operated"
                        },
                        "segments": [
                            {
                                "id": "14581-13870-2410271235-2410271435--32222",
                                "origin": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "destination": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "departure": "2024-10-27T12:35:00",
                                "arrival": "2024-10-27T14:35:00",
                                "durationInMinutes": 120,
                                "flightNumber": "1212",
                                "marketingCarrier": {
                                    "id": -32222,
                                    "name": "Iberia",
                                    "alternateId": "IB",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32639,
                                    "name": "Air Nostrum",
                                    "alternateId": "YW",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    }
                ],
                "isSelfTransfer": False,
                "isProtectedSelfTransfer": False,
                "farePolicy": {
                    "isChangeAllowed": False,
                    "isPartiallyChangeable": False,
                    "isCancellationAllowed": False,
                    "isPartiallyRefundable": False
                },
                "eco": {
                    "ecoContenderDelta": 27.515114
                },
                "fareAttributes": {},
                "isMashUp": False,
                "hasFlexibleOptions": False,
                "score": 0.767222
            },
            {
                "id": "13870-2410241720--32222-0-14581-2410241915|14581-2410271235--32222-0-13870-2410271435",
                "price": {
                    "raw": 2237.73,
                    "formatted": "2,238 €",
                    "pricingOptionId": "LseyH0uNk3cS"
                },
                "legs": [
                    {
                        "id": "13870-2410241720--32222-0-14581-2410241915",
                        "origin": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 115,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-24T17:20:00",
                        "arrival": "2024-10-24T19:15:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32222,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/IB.png",
                                    "name": "Iberia"
                                }
                            ],
                            "operating": [
                                {
                                    "id": -32639,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/YW.png",
                                    "name": "Air Nostrum"
                                }
                            ],
                            "operationType": "not_operated"
                        },
                        "segments": [
                            {
                                "id": "13870-14581-2410241720-2410241915--32222",
                                "origin": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "destination": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "departure": "2024-10-24T17:20:00",
                                "arrival": "2024-10-24T19:15:00",
                                "durationInMinutes": 115,
                                "flightNumber": "8730",
                                "marketingCarrier": {
                                    "id": -32222,
                                    "name": "Iberia",
                                    "alternateId": "IB",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32639,
                                    "name": "Air Nostrum",
                                    "alternateId": "YW",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    },
                    {
                        "id": "14581-2410271235--32222-0-13870-2410271435",
                        "origin": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 120,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-27T12:35:00",
                        "arrival": "2024-10-27T14:35:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32222,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/IB.png",
                                    "name": "Iberia"
                                }
                            ],
                            "operating": [
                                {
                                    "id": -32639,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/YW.png",
                                    "name": "Air Nostrum"
                                }
                            ],
                            "operationType": "not_operated"
                        },
                        "segments": [
                            {
                                "id": "14581-13870-2410271235-2410271435--32222",
                                "origin": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "destination": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "departure": "2024-10-27T12:35:00",
                                "arrival": "2024-10-27T14:35:00",
                                "durationInMinutes": 120,
                                "flightNumber": "1212",
                                "marketingCarrier": {
                                    "id": -32222,
                                    "name": "Iberia",
                                    "alternateId": "IB",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32639,
                                    "name": "Air Nostrum",
                                    "alternateId": "YW",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    }
                ],
                "isSelfTransfer": False,
                "isProtectedSelfTransfer": False,
                "farePolicy": {
                    "isChangeAllowed": False,
                    "isPartiallyChangeable": False,
                    "isCancellationAllowed": False,
                    "isPartiallyRefundable": False
                },
                "eco": {
                    "ecoContenderDelta": 27.515114
                },
                "fareAttributes": {},
                "isMashUp": False,
                "hasFlexibleOptions": False,
                "score": 0.729379
            },
            {
                "id": "13870-2410241720--31685-1-14581-2410242115|14581-2410271430--32356-0-13870-2410271630",
                "price": {
                    "raw": 1195.16,
                    "formatted": "1,196 €",
                    "pricingOptionId": "rQ2K6YAU6PbW"
                },
                "legs": [
                    {
                        "id": "13870-2410241720--31685-1-14581-2410242115",
                        "origin": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 235,
                        "stopCount": 1,
                        "isSmallestStops": False,
                        "departure": "2024-10-24T17:20:00",
                        "arrival": "2024-10-24T21:15:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -31685,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/07.png",
                                    "name": "Vueling Airlines"
                                }
                            ],
                            "operationType": "fully_operated"
                        },
                        "segments": [
                            {
                                "id": "13870-9772-2410241720-2410241840--31685",
                                "origin": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "destination": {
                                    "flightPlaceId": "BCN",
                                    "displayCode": "BCN",
                                    "parent": {
                                        "flightPlaceId": "BARC",
                                        "displayCode": "BCN",
                                        "name": "Barcelona",
                                        "type": "City"
                                    },
                                    "name": "Barcelona",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "departure": "2024-10-24T17:20:00",
                                "arrival": "2024-10-24T18:40:00",
                                "durationInMinutes": 80,
                                "flightNumber": "1007",
                                "marketingCarrier": {
                                    "id": -31685,
                                    "name": "Vueling Airlines",
                                    "alternateId": "07",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -31685,
                                    "name": "Vueling Airlines",
                                    "alternateId": "07",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            },
                            {
                                "id": "9772-14581-2410241955-2410242115--31685",
                                "origin": {
                                    "flightPlaceId": "BCN",
                                    "displayCode": "BCN",
                                    "parent": {
                                        "flightPlaceId": "BARC",
                                        "displayCode": "BCN",
                                        "name": "Barcelona",
                                        "type": "City"
                                    },
                                    "name": "Barcelona",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "destination": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "departure": "2024-10-24T19:55:00",
                                "arrival": "2024-10-24T21:15:00",
                                "durationInMinutes": 80,
                                "flightNumber": "1517",
                                "marketingCarrier": {
                                    "id": -31685,
                                    "name": "Vueling Airlines",
                                    "alternateId": "07",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -31685,
                                    "name": "Vueling Airlines",
                                    "alternateId": "07",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    },
                    {
                        "id": "14581-2410271430--32356-0-13870-2410271630",
                        "origin": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 120,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-27T14:30:00",
                        "arrival": "2024-10-27T16:30:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32356,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/EZ.png",
                                    "name": "easyJet"
                                }
                            ],
                            "operationType": "fully_operated"
                        },
                        "segments": [
                            {
                                "id": "14581-13870-2410271430-2410271630--32356",
                                "origin": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "destination": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "departure": "2024-10-27T14:30:00",
                                "arrival": "2024-10-27T16:30:00",
                                "durationInMinutes": 120,
                                "flightNumber": "1703",
                                "marketingCarrier": {
                                    "id": -32356,
                                    "name": "easyJet",
                                    "alternateId": "EZ",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32356,
                                    "name": "easyJet",
                                    "alternateId": "EZ",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    }
                ],
                "isSelfTransfer": False,
                "isProtectedSelfTransfer": False,
                "farePolicy": {
                    "isChangeAllowed": False,
                    "isPartiallyChangeable": False,
                    "isCancellationAllowed": False,
                    "isPartiallyRefundable": False
                },
                "eco": {
                    "ecoContenderDelta": 28.495901
                },
                "fareAttributes": {},
                "tags": [
                    "third_cheapest"
                ],
                "isMashUp": True,
                "hasFlexibleOptions": False,
                "score": 0.70914
            },
            {
                "id": "13870-2410242210--32222-0-14581-2410250005|14581-2410271235--32222-0-13870-2410271435",
                "price": {
                    "raw": 1994.81,
                    "formatted": "1,995 €",
                    "pricingOptionId": "MxWawwQSXMsV"
                },
                "legs": [
                    {
                        "id": "13870-2410242210--32222-0-14581-2410250005",
                        "origin": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 115,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-24T22:10:00",
                        "arrival": "2024-10-25T00:05:00",
                        "timeDeltaInDays": 1,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32222,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/IB.png",
                                    "name": "Iberia"
                                }
                            ],
                            "operating": [
                                {
                                    "id": -32639,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/YW.png",
                                    "name": "Air Nostrum"
                                }
                            ],
                            "operationType": "not_operated"
                        },
                        "segments": [
                            {
                                "id": "13870-14581-2410242210-2410250005--32222",
                                "origin": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "destination": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "departure": "2024-10-24T22:10:00",
                                "arrival": "2024-10-25T00:05:00",
                                "durationInMinutes": 115,
                                "flightNumber": "8732",
                                "marketingCarrier": {
                                    "id": -32222,
                                    "name": "Iberia",
                                    "alternateId": "IB",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32639,
                                    "name": "Air Nostrum",
                                    "alternateId": "YW",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    },
                    {
                        "id": "14581-2410271235--32222-0-13870-2410271435",
                        "origin": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 120,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-27T12:35:00",
                        "arrival": "2024-10-27T14:35:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32222,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/IB.png",
                                    "name": "Iberia"
                                }
                            ],
                            "operating": [
                                {
                                    "id": -32639,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/YW.png",
                                    "name": "Air Nostrum"
                                }
                            ],
                            "operationType": "not_operated"
                        },
                        "segments": [
                            {
                                "id": "14581-13870-2410271235-2410271435--32222",
                                "origin": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "destination": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "departure": "2024-10-27T12:35:00",
                                "arrival": "2024-10-27T14:35:00",
                                "durationInMinutes": 120,
                                "flightNumber": "1212",
                                "marketingCarrier": {
                                    "id": -32222,
                                    "name": "Iberia",
                                    "alternateId": "IB",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32639,
                                    "name": "Air Nostrum",
                                    "alternateId": "YW",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    }
                ],
                "isSelfTransfer": False,
                "isProtectedSelfTransfer": False,
                "farePolicy": {
                    "isChangeAllowed": False,
                    "isPartiallyChangeable": False,
                    "isCancellationAllowed": False,
                    "isPartiallyRefundable": False
                },
                "eco": {
                    "ecoContenderDelta": 27.515114
                },
                "fareAttributes": {},
                "isMashUp": False,
                "hasFlexibleOptions": False,
                "score": 0.707598
            },
            {
                "id": "13870-2410240925--31685,-32356-1-14581-2410241455|14581-2410271430--32356-0-13870-2410271630",
                "price": {
                    "raw": 1044.43,
                    "formatted": "1,045 €",
                    "pricingOptionId": "gN2YnwQ9Emoy"
                },
                "legs": [
                    {
                        "id": "13870-2410240925--31685,-32356-1-14581-2410241455",
                        "origin": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 330,
                        "stopCount": 1,
                        "isSmallestStops": False,
                        "departure": "2024-10-24T09:25:00",
                        "arrival": "2024-10-24T14:55:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -31685,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/07.png",
                                    "name": "Vueling Airlines"
                                },
                                {
                                    "id": -32356,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/EZ.png",
                                    "name": "easyJet"
                                }
                            ],
                            "operationType": "fully_operated"
                        },
                        "segments": [
                            {
                                "id": "13870-9772-2410240925-2410241045--31685",
                                "origin": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "destination": {
                                    "flightPlaceId": "BCN",
                                    "displayCode": "BCN",
                                    "parent": {
                                        "flightPlaceId": "BARC",
                                        "displayCode": "BCN",
                                        "name": "Barcelona",
                                        "type": "City"
                                    },
                                    "name": "Barcelona",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "departure": "2024-10-24T09:25:00",
                                "arrival": "2024-10-24T10:45:00",
                                "durationInMinutes": 80,
                                "flightNumber": "1003",
                                "marketingCarrier": {
                                    "id": -31685,
                                    "name": "Vueling Airlines",
                                    "alternateId": "07",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -31685,
                                    "name": "Vueling Airlines",
                                    "alternateId": "07",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            },
                            {
                                "id": "9772-14581-2410241335-2410241455--32356",
                                "origin": {
                                    "flightPlaceId": "BCN",
                                    "displayCode": "BCN",
                                    "parent": {
                                        "flightPlaceId": "BARC",
                                        "displayCode": "BCN",
                                        "name": "Barcelona",
                                        "type": "City"
                                    },
                                    "name": "Barcelona",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "destination": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "departure": "2024-10-24T13:35:00",
                                "arrival": "2024-10-24T14:55:00",
                                "durationInMinutes": 80,
                                "flightNumber": "1708",
                                "marketingCarrier": {
                                    "id": -32356,
                                    "name": "easyJet",
                                    "alternateId": "EZ",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32356,
                                    "name": "easyJet",
                                    "alternateId": "EZ",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    },
                    {
                        "id": "14581-2410271430--32356-0-13870-2410271630",
                        "origin": {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice",
                            "displayCode": "NCE",
                            "city": "Nice",
                            "country": "France",
                            "isHighlighted": False
                        },
                        "destination": {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid",
                            "displayCode": "MAD",
                            "city": "Madrid",
                            "country": "Spain",
                            "isHighlighted": False
                        },
                        "durationInMinutes": 120,
                        "stopCount": 0,
                        "isSmallestStops": False,
                        "departure": "2024-10-27T14:30:00",
                        "arrival": "2024-10-27T16:30:00",
                        "timeDeltaInDays": 0,
                        "carriers": {
                            "marketing": [
                                {
                                    "id": -32356,
                                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/EZ.png",
                                    "name": "easyJet"
                                }
                            ],
                            "operationType": "fully_operated"
                        },
                        "segments": [
                            {
                                "id": "14581-13870-2410271430-2410271630--32356",
                                "origin": {
                                    "flightPlaceId": "NCE",
                                    "displayCode": "NCE",
                                    "parent": {
                                        "flightPlaceId": "NICE",
                                        "displayCode": "NCE",
                                        "name": "Nice",
                                        "type": "City"
                                    },
                                    "name": "Nice",
                                    "type": "Airport",
                                    "country": "France"
                                },
                                "destination": {
                                    "flightPlaceId": "MAD",
                                    "displayCode": "MAD",
                                    "parent": {
                                        "flightPlaceId": "MADR",
                                        "displayCode": "MAD",
                                        "name": "Madrid",
                                        "type": "City"
                                    },
                                    "name": "Madrid",
                                    "type": "Airport",
                                    "country": "Spain"
                                },
                                "departure": "2024-10-27T14:30:00",
                                "arrival": "2024-10-27T16:30:00",
                                "durationInMinutes": 120,
                                "flightNumber": "1703",
                                "marketingCarrier": {
                                    "id": -32356,
                                    "name": "easyJet",
                                    "alternateId": "EZ",
                                    "allianceId": 0,
                                    "displayCode": ""
                                },
                                "operatingCarrier": {
                                    "id": -32356,
                                    "name": "easyJet",
                                    "alternateId": "EZ",
                                    "allianceId": 0,
                                    "displayCode": ""
                                }
                            }
                        ]
                    }
                ],
                "isSelfTransfer": False,
                "isProtectedSelfTransfer": True,
                "farePolicy": {
                    "isChangeAllowed": False,
                    "isPartiallyChangeable": False,
                    "isCancellationAllowed": False,
                    "isPartiallyRefundable": False
                },
                "eco": {
                    "ecoContenderDelta": 29.226702
                },
                "fareAttributes": {},
                "tags": [
                    "second_cheapest"
                ],
                "isMashUp": False,
                "hasFlexibleOptions": False,
                "score": 0.680028
            }
        ],
        "messages": [],
        "filterStats": {
            "duration": {
                "min": 120,
                "max": 380,
                "multiCityMin": 235,
                "multiCityMax": 500
            },
            "airports": [
                {
                    "city": "Madrid",
                    "airports": [
                        {
                            "id": "MAD",
                            "entityId": "95565077",
                            "name": "Madrid"
                        }
                    ]
                },
                {
                    "city": "Nice",
                    "airports": [
                        {
                            "id": "NCE",
                            "entityId": "95673860",
                            "name": "Nice"
                        }
                    ]
                }
            ],
            "carriers": [
                {
                    "id": -32356,
                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/EZ.png",
                    "name": "easyJet"
                },
                {
                    "id": -32222,
                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/IB.png",
                    "name": "Iberia"
                },
                {
                    "id": -31685,
                    "logoUrl": "https://logos.skyscnr.com/images/airlines/favicon/07.png",
                    "name": "Vueling Airlines"
                }
            ],
            "stopPrices": {
                "direct": {
                    "isPresent": True,
                    "formattedPrice": "1,554 €"
                },
                "one": {
                    "isPresent": True,
                    "formattedPrice": "941 €"
                },
                "twoOrMore": {
                    "isPresent": False
                }
            }
        },
        "flightsSessionId": "f84ae906-6a68-4360-9a6a-844ebb077508",
        "destinationImageUrl": "https://content.skyscnr.com/m/3719e8f4a5daf43d/original/Flights-Placeholder.jpg"
    }
}

 para ida y vuelta un for para 1 y 2 (ida es 1 y 2 es la vuelta)

In [21]:
len(results["data"]["itineraries"])

10

In [92]:
clase = querystrings[0]["cabinClass"]

'economy'

In [93]:
df_plane = pd.DataFrame()
for i in range(0,len(results["data"]["itineraries"])):
    precio = results["data"]["itineraries"][i]["price"]["formatted"]
    origen_ida = results["data"]["itineraries"][i]["legs"][0]["origin"]["city"]
    destino_ida = results["data"]["itineraries"][i]["legs"][0]["destination"]["city"]
    duracion_ida = results["data"]["itineraries"][i]["legs"][0]["durationInMinutes"]
    fecha_salida_ida = results["data"]["itineraries"][i]["legs"][0]["departure"]
    fecha_llegada_ida = results["data"]["itineraries"][i]["legs"][0]["arrival"]
    aerolinea_ida = results["data"]["itineraries"][i]["legs"][0]["carriers"]["marketing"][0]["name"]

    origen_vuelta = results["data"]["itineraries"][i]["legs"][1]["origin"]["city"]
    destino_vuelta = results["data"]["itineraries"][i]["legs"][1]["destination"]["city"]
    duracion_vuelta = results["data"]["itineraries"][i]["legs"][1]["durationInMinutes"]
    fecha_salida_vuelta = results["data"]["itineraries"][i]["legs"][1]["departure"]
    fecha_llegada_vuelta = results["data"]["itineraries"][i]["legs"][1]["arrival"]
    aerolinea_vuelta = results["data"]["itineraries"][i]["legs"][1]["carriers"]["marketing"][0]["name"]
    df_temp = pd.DataFrame({
        "clase" : clase,
        "precio (€)" : [precio],
        "origen_ida" : [origen_ida],
        "destino_ida" : [destino_ida],
        "duracion_ida (min)" :[ duracion_ida],
        "fecha_salida_ida" : [fecha_salida_ida],
        "fecha_llegada_ida" : [fecha_llegada_ida],
        "aerolínea_ida" : [aerolinea_ida],
        "origen_vuelta" : [origen_vuelta],
        "destino_vuelta" : [destino_vuelta],
        "duracion_vuelta (min)" : [duracion_vuelta],
        "fecha_salida_vuelta" : [fecha_salida_vuelta],
        "fecha_llegada_vuelta" : [fecha_llegada_vuelta],
        "aerolínea_vuelta" : [aerolinea_vuelta]
    })
    df_temp["precio (€)"] = df_temp["precio (€)"].str.strip(" €").str.replace(",","").astype(int)
    df_temp["fecha_salida_ida"] = pd.to_datetime(df_temp["fecha_salida_ida"],format="mixed")
    df_temp["fecha_llegada_ida"] = pd.to_datetime(df_temp["fecha_llegada_ida"],format="mixed")
    df_temp["fecha_salida_vuelta"] = pd.to_datetime(df_temp["fecha_salida_vuelta"],format="mixed")
    df_temp["fecha_llegada_vuelta"] = pd.to_datetime(df_temp["fecha_llegada_vuelta"],format="mixed")
    df_temp["duracion_ida (min)"] = df_temp["duracion_ida (min)"].astype(int)
    df_temp["duracion_vuelta (min)"] = df_temp["duracion_vuelta (min)"].astype(int)
    df_plane = pd.concat([df_plane,df_temp],ignore_index=True)

df_plane
#origen

,clase,precio (€),origen_ida,destino_ida,duracion_ida (min),fecha_salida_ida,fecha_llegada_ida,aerolínea_ida,origen_vuelta,destino_vuelta,duracion_vuelta (min),fecha_salida_vuelta,fecha_llegada_vuelta,aerolínea_vuelta
0,first,1970,Madrid,Nice,115,2024-10-24 10:15:00,2024-10-24 12:10:00,Iberia,Nice,Madrid,120,2024-10-27 19:00:00,2024-10-27 21:00:00,Iberia
1,first,2036,Madrid,Nice,115,2024-10-24 17:20:00,2024-10-24 19:15:00,Iberia,Nice,Madrid,120,2024-10-27 19:00:00,2024-10-27 21:00:00,Iberia
2,first,941,Madrid,Nice,380,2024-10-24 09:25:00,2024-10-24 15:45:00,Vueling Airlines,Nice,Madrid,120,2024-10-27 14:30:00,2024-10-27 16:30:00,easyJet
3,first,1554,Madrid,Nice,115,2024-10-24 22:10:00,2024-10-25 00:05:00,Iberia,Nice,Madrid,120,2024-10-27 14:30:00,2024-10-27 16:30:00,easyJet
4,first,1793,Madrid,Nice,115,2024-10-24 22:10:00,2024-10-25 00:05:00,Iberia,Nice,Madrid,120,2024-10-27 19:00:00,2024-10-27 21:00:00,Iberia
5,first,2172,Madrid,Nice,115,2024-10-24 10:15:00,2024-10-24 12:10:00,Iberia,Nice,Madrid,120,2024-10-27 12:35:00,2024-10-27 14:35:00,Iberia
6,first,2238,Madrid,Nice,115,2024-10-24 17:20:00,2024-10-24 19:15:00,Iberia,Nice,Madrid,120,2024-10-27 12:35:00,2024-10-27 14:35:00,Iberia
7,first,1196,Madrid,Nice,235,2024-10-24 17:20:00,2024-10-24 21:15:00,Vueling Airlines,Nice,Madrid,120,2024-10-27 14:30:00,2024-10-27 16:30:00,easyJet
8,first,1995,Madrid,Nice,115,2024-10-24 22:10:00,2024-10-25 00:05:00,Iberia,Nice,Madrid,120,2024-10-27 12:35:00,2024-10-27 14:35:00,Iberia
9,first,1045,Madrid,Nice,330,2024-10-24 09:25:00,2024-10-24 14:55:00,Vueling Airlines,Nice,Madrid,120,2024-10-27 14:30:00,2024-10-27 16:30:00,easyJet
